### Clean acquire

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy.pool import StaticPool
from sqlalchemy import create_engine

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
path = '../data/raw/raw_data_project_m1.db'
def getall(path):
    print('connecting...')
    engine = create_engine(f'sqlite:///{path}', poolclass=StaticPool)
    header_tables = engine.table_names()
    for table in range(0, len(header_tables)):
        single = header_tables[table]
        query = f'SELECT * FROM {single}'
        data = pd.read_sql(query, con=engine)
        if table == 0:
            raw_data = data
        else:
            raw_data = pd.merge(raw_data, data, how='left', on='uuid')
    print('receiving...')
    raw_data.to_csv('data/raw/raw_data_compile.csv', index=False)
    print('copy saved')
    return raw_data

### La función devuelve el csv

In [4]:
all = pd.read_csv('../data/raw/raw_data_compile.csv')

In [5]:
all

,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst
0,car,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working
2,83127080-da3d-0133-c74f-0a81e8b09a82,NaN,no,NaN,AT,city,32 years old,male,nO,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...
3,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,Country,45 years old,Male,YES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above
4,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,city,41 years old,Fem,yES,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9644,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,urban,37 years old,FeMale,nO,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance
9645,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,urban,53 years old,Male,yES,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...
9646,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,NaN,SK,Non-Rural,1992,male,NO,juvenile,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above
9647,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,city,47 years old,male,yES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...


### 1er paso Web_scrapping de Paises

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
def standardizeCountry(all):
    print('starting with country standardize...')
    url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    frame = soup.find_all('td')
    countries = [item.text.replace('\n', '').replace(' ', '') for item in frame]
    countries_clean = list(filter(None, countries))
    row_split = 2
    rows_refactored = [countries_clean[x:x + row_split] for x in range(0, len(countries_clean), row_split)]
    df = pd.DataFrame(rows_refactored, columns=['Country', 'country_code'])
    df['country_code'] = df['country_code'].str.strip('()').astype(str)
    all['country_code'] = all['country_code'].replace({'GR': 'EL',
                                                       'GB': 'UK'})
    allone = pd.merge(all, df, on='country_code')
    print('receiving countries standardized...')
    return allone

In [8]:
standardizeCountry(all)

starting with country standardize...
receiving countries standardized...


,uuid,dem_education_level,dem_full_time_job,normalized_job_code,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Country
0,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,861a9b9151e11362eb3c77ca914172d0,AT,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Austria
1,83127080-da3d-0133-c74f-0a81e8b09a82,NaN,no,NaN,AT,city,32 years old,male,nO,26_39,I have heard just a little about it,I would not vote,‰Û_ gain additional skills,It creates more equality of opportunity,Foreigners might come to my country and take a...,Austria
2,15626d40-db13-0133-ea5c-0a81e8b09a82,high,yes,049a3f3a2b5f85cb2971ba77ad66e10c,AT,Country,45 years old,Male,YES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ work less,It reduces anxiety about financing basic needs,None of the above,Austria
3,24954a70-db98-0133-4a64-0a81e8b09a82,high,yes,f4b2fb1aa40f661488e2782b6d57ad2f,AT,city,41 years old,Fem,yES,40_65,I have heard just a little about it,I would probably vote for it,None of the above,It reduces anxiety about financing basic needs,It is impossible to finance | It might encoura...,Austria
4,28583060-dbe2-0133-ff9c-0a81e8b09a82,high,yes,27af8700f5577cec835acee2cb90a2ff,AT,countryside,1990,FeMale,NO,26_39,I understand it fully,I would vote against it,A basic income would not affect my work choices,It encourages financial independence and self-...,It is impossible to finance | It might encoura...,Austria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,7d1ac020-dcb4-0133-817a-0a81e8b09a82,high,yes,847165cfda6b1dc82ae22b967da8af2f,SK,urban,37 years old,FeMale,nO,26_39,I understand it fully,I would probably vote for it,‰Û_ spend more time with my family,It reduces bureaucracy and administrative expe...,It is impossible to finance,Slovakia
9644,39f989f0-db52-0133-8482-0a81e8b09a82,high,yes,a4d5b8b38f9513825d0d94a981ebe962,SK,urban,53 years old,Male,yES,40_65,I have never heard of it,I would probably vote against it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,It might encourage people to stop working | On...,Slovakia
9645,70ce4a90-d965-0133-f5e4-0a81e8b09a82,low,no,NaN,SK,Non-Rural,1992,male,NO,juvenile,I have heard just a little about it,I would not vote,‰Û_ spend more time with my family,It reduces anxiety about financing basic needs,None of the above,Slovakia
9646,2896e440-db3c-0133-5b67-0a81e8b09a82,low,yes,775190277a849cba701b306a7b374c0a,SK,city,47 years old,male,yES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces bureaucracy and administrative expe...,Foreigners might come to my country and take a...,Slovakia


In [9]:
countries = standardizeCountry(all)

starting with country standardize...
receiving countries standardized...


In [10]:
country_lst = countries['Country'].unique()
country_lst

array(['Austria', 'Belgium', 'Bulgaria', 'Cyprus', 'Czechia', 'Germany',
       'Denmark', 'Estonia', 'Spain', 'Finland', 'France',
       'UnitedKingdom', 'Greece', 'Croatia', 'Hungary', 'Ireland',
       'Italy', 'Lithuania', 'Luxembourg', 'Latvia', 'Malta',
       'Netherlands', 'Poland', 'Portugal', 'Romania', 'Sweden',
       'Slovenia', 'Slovakia'], dtype=object)

### 2do paso Obtener los nombres de los trabajos

In [11]:
def getjobtitle(allone):
    print('starting with job standardize...')
    jobs = allone['normalized_job_code'].unique()
    job_dict = []
    for id in jobs:
        response = requests.get(f'http://api.dataatwork.org/v1/jobs/{id}')
        job_dict.append(response.json())
    api_df = pd.DataFrame(job_dict)
    rename_api_df = api_df.rename(columns={'uuid': 'normalized_job_code',
                                               'title': 'Job_Title'})
    sort_df = rename_api_df.set_index('normalized_job_code')
    alltwo = sort_df.join(allone.set_index('normalized_job_code'), on='normalized_job_code').reset_index()
    alltwo['Job_Title'].fillna('unemployed', inplace=True)
    print('receiving jobs standardized...')
    return alltwo

In [12]:
allthree = getjobtitle(standardizeCountry(all))

starting with country standardize...
receiving countries standardized...
starting with job standardize...
receiving jobs standardized...


In [13]:
allthree.head()

,normalized_job_code,Job_Title,normalized_job_title,parent_uuid,error,uuid,dem_education_level,dem_full_time_job,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Country
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,AT,urban,57 years old,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Austria
1,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,cf24ed60-da3f-0133-0034-0a81e8b09a82,high,yes,AT,Country,48 years old,FeMale,NO,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,Austria
2,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,9ffc7390-dba5-0133-259b-0a81e8b09a82,high,yes,BE,Country,43 years old,male,YES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,Belgium
3,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,a6361230-da58-0133-cd5a-0a81e8b09a82,medium,yes,BE,urban,43 years old,Male,yES,40_65,I understand it fully,I would probably vote against it,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,Belgium
4,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,1e2b4750-d987-0133-3c7c-0a81e8b09a82,high,yes,BG,urban,52 years old,female,YES,40_65,I have heard just a little about it,I would probably vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,Bulgaria


In [14]:
print('DATAFRAME SHAPE \n - rows: {} \n - columns: {}'.format(allthree.shape[0],allthree.shape[1]),'\n')
print('DATAFRAME NULLS COUNT: \n{}'.format(allthree.isnull().sum()),'\n')
print('DATAFRAME COLUMNS TYPE: \n{}'.format(allthree.dtypes,'\n'))

DATAFRAME SHAPE 
 - rows: 9648 
 - columns: 20 

DATAFRAME NULLS COUNT: 
normalized_job_code                                    3946
Job_Title                                                 0
normalized_job_title                                   3946
parent_uuid                                            3946
error                                                  5702
uuid                                                      0
dem_education_level                                     663
dem_full_time_job                                         0
country_code                                              0
rural                                                     0
age                                                       0
gender                                                    0
dem_has_children                                          0
age_group                                                 0
question_bbi_2016wave4_basicincome_awareness              0
question_bbi_2016wave4_basi

### 3er paso estandarizar la edad

In [15]:
def calculateAge(birthDate):
    today = pd.Timestamp('2016').year
    each_age = today-int(birthDate)
    return each_age

In [16]:
def realAge(allthree):
    print('starting wit age standardize...')
    allthree['age'] = allthree['age'].str.replace('\D', '')
    age = []
    for birth in allthree['age']:
        if len(birth) == 4:
            age.append(calculateAge(birth))
        else:
            age.append(birth)
    allthree['age'] = np.array(age)
    print('receiving age standardized...')
    return allthree

In [17]:
realAge(allthree)

starting wit age standardize...
receiving age standardized...


,normalized_job_code,Job_Title,normalized_job_title,parent_uuid,error,uuid,dem_education_level,dem_full_time_job,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Country
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,AT,urban,57,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Austria
1,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,cf24ed60-da3f-0133-0034-0a81e8b09a82,high,yes,AT,Country,48,FeMale,NO,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,Austria
2,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,9ffc7390-dba5-0133-259b-0a81e8b09a82,high,yes,BE,Country,43,male,YES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,Belgium
3,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,a6361230-da58-0133-cd5a-0a81e8b09a82,medium,yes,BE,urban,43,Male,yES,40_65,I understand it fully,I would probably vote against it,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,Belgium
4,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,1e2b4750-d987-0133-3c7c-0a81e8b09a82,high,yes,BG,urban,52,female,YES,40_65,I have heard just a little about it,I would probably vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,Bulgaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,high,yes,IT,Country,58,male,yES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ look for a different job,It reduces anxiety about financing basic needs...,Only the people who need it most should get so...,Italy
9644,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,c8c33390-da69-0133-063a-0a81e8b09a82,medium,yes,PL,city,44,male,YES,40_65,I have heard just a little about it,I would vote for it,‰Û_ look for a different job,It reduces anxiety about financing basic needs,It might encourage people to stop working | It...,Poland
9645,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,d27d24d0-d9b1-0133-03d4-0a81e8b09a82,high,yes,PL,city,24,male,nO,14_25,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,Poland
9646,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,529f3080-d99a-0133-1b7b-0a81e8b09a82,no,yes,PT,city,40,male,yES,40_65,I know something about it,I would probably vote for it,‰Û_ do more volunteering work,It encourages financial independence and self-...,It is impossible to finance,Portugal


In [18]:
allfour = realAge(allthree)

starting wit age standardize...
receiving age standardized...


### Analysing

In [37]:
def grouptable(allfour):
    print('selecting columns...')
    first_table = allfour[['Country','Job_Title','age','uuid']]
    print('making some calculations')
    first_table.rename(columns={'uuid':'Quantity'}, inplace=True)
    series_group = first_table.groupby(['Country','Job_Title','age'])['Quantity'].count()
    df_grouped = pd.DataFrame(series_group).reset_index()
    df_grouped['Percentage'] = df_grouped['Quantity'] / df_grouped['Quantity'].sum()*1000
    print('receiving columns...')
    return df_grouped

In [38]:
df_grouped = grouptable(allfour)

selecting columns...
making some calculations
receiving columns...


In [39]:
df_grouped

,Country,Job_Title,age,Quantity,Percentage
0,Austria,Automatic Data Processing Customer Liaison (AD...,31,1,0.103648
1,Austria,Automatic Data Processing Customer Liaison (AD...,40,1,0.103648
2,Austria,Automatic Data Processing Planner,48,1,0.103648
3,Austria,Automatic Data Processing Planner,57,1,0.103648
4,Austria,Automatic Data Processing Systems Security Spe...,27,1,0.103648
...,...,...,...,...,...
6135,UnitedKingdom,unemployed,61,4,0.414594
6136,UnitedKingdom,unemployed,62,10,1.036484
6137,UnitedKingdom,unemployed,63,9,0.932836
6138,UnitedKingdom,unemployed,64,6,0.621891


### The final function

In [ ]:
def grouptable(moreall, country):
    print('selecting columns...')
    first_table = moreall[['Country','Job_Title','age','uuid']]
    print('making some calculations')
    first_table.rename(columns={'uuid':'Quantity'}, inplace=True)
    series_group = first_table.groupby(['Country','Job_Title','age'])['Quantity'].count()
    df_grouped = pd.DataFrame(series_group).reset_index()
    df_grouped['Percentage'] = df_grouped['Quantity']/df_grouped['Quantity'].sum()*1000
    print('receiving columns...')
    if country == 'all':
        print(df_grouped)
        print('No country filter. Exported results to csv')
        df_grouped.to_csv('data/results/analysed_age_info.csv', index=False)
    else:
        df_grouped = df_grouped[df_grouped['Country'] == country]
        print(df_grouped)
        print(f'Filtered by {country} and exported to csv')
        df_grouped.to_csv(f'data/results/{country}_analysed_rural_info.csv', index=False)
    return df_grouped


,normalized_job_code,Job_Title,normalized_job_title,parent_uuid,error,uuid,dem_education_level,dem_full_time_job,country_code,rural,age,gender,dem_has_children,age_group,question_bbi_2016wave4_basicincome_awareness,question_bbi_2016wave4_basicincome_vote,question_bbi_2016wave4_basicincome_effect,question_bbi_2016wave4_basicincome_argumentsfor,question_bbi_2016wave4_basicincome_argumentsagainst,Country
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,54f0f1c0-dda1-0133-a559-0a81e8b09a82,high,yes,AT,urban,57,male,yES,40_65,I understand it fully,I would probably vote for it,A basic income would not affect my work choices,It increases appreciation for household work a...,It might encourage people to stop working,Austria
1,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,cf24ed60-da3f-0133-0034-0a81e8b09a82,high,yes,AT,Country,48,FeMale,NO,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,Austria
2,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,9ffc7390-dba5-0133-259b-0a81e8b09a82,high,yes,BE,Country,43,male,YES,40_65,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,It increases dependence on the state,Belgium
3,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,a6361230-da58-0133-cd5a-0a81e8b09a82,medium,yes,BE,urban,43,Male,yES,40_65,I understand it fully,I would probably vote against it,A basic income would not affect my work choices,None of the above,Only the people who need it most should get so...,Belgium
4,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner,automatic data processing planner,0148f61d4227497728ce33490843d056,NaN,1e2b4750-d987-0133-3c7c-0a81e8b09a82,high,yes,BG,urban,52,female,YES,40_65,I have heard just a little about it,I would probably vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs,It might encourage people to stop working,Bulgaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9643,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,1cb1aac0-d94c-0133-8baa-0a81e8b09a82,high,yes,IT,Country,58,male,yES,40_65,I have heard just a little about it,I would probably vote for it,‰Û_ look for a different job,It reduces anxiety about financing basic needs...,Only the people who need it most should get so...,Italy
9644,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,c8c33390-da69-0133-063a-0a81e8b09a82,medium,yes,PL,city,44,male,YES,40_65,I have heard just a little about it,I would vote for it,‰Û_ look for a different job,It reduces anxiety about financing basic needs,It might encourage people to stop working | It...,Poland
9645,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,d27d24d0-d9b1-0133-03d4-0a81e8b09a82,high,yes,PL,city,24,male,nO,14_25,I understand it fully,I would vote for it,A basic income would not affect my work choices,It reduces anxiety about financing basic needs...,None of the above,Poland
9646,4cee16550636e292b8d136486fce943b,Data Capture Clerk,data capture clerk,bf88358c56bb6cbe7eabec38ae333d19,NaN,529f3080-d99a-0133-1b7b-0a81e8b09a82,no,yes,PT,city,40,male,yES,40_65,I know something about it,I would probably vote for it,‰Û_ do more volunteering work,It encourages financial independence and self-...,It is impossible to finance,Portugal


In [73]:
allfour_reduce = allfour[['Country','Job_Title','age','uuid']]

In [74]:
allfour_reduce

,Country,Job_Title,age,uuid
0,Austria,Automatic Data Processing Planner,57,54f0f1c0-dda1-0133-a559-0a81e8b09a82
1,Austria,Automatic Data Processing Planner,48,cf24ed60-da3f-0133-0034-0a81e8b09a82
2,Belgium,Automatic Data Processing Planner,43,9ffc7390-dba5-0133-259b-0a81e8b09a82
3,Belgium,Automatic Data Processing Planner,43,a6361230-da58-0133-cd5a-0a81e8b09a82
4,Bulgaria,Automatic Data Processing Planner,52,1e2b4750-d987-0133-3c7c-0a81e8b09a82
...,...,...,...,...
9643,Italy,Data Capture Clerk,58,1cb1aac0-d94c-0133-8baa-0a81e8b09a82
9644,Poland,Data Capture Clerk,44,c8c33390-da69-0133-063a-0a81e8b09a82
9645,Poland,Data Capture Clerk,24,d27d24d0-d9b1-0133-03d4-0a81e8b09a82
9646,Portugal,Data Capture Clerk,40,529f3080-d99a-0133-1b7b-0a81e8b09a82


In [75]:
allfour_reduce.groupby(['Country', 'Job_Title','age'])['uuid'].count()

Country        Job_Title                                                                     age
Austria        Automatic Data Processing Customer Liaison (ADP Customer Liaison)             31      1
                                                                                             40      1
               Automatic Data Processing Planner                                             48      1
                                                                                             57      1
               Automatic Data Processing Systems Security Specialist (ADP Systems Security)  27      1
                                                                                                    ..
UnitedKingdom  unemployed                                                                    61      4
                                                                                             62     10
                                                                               

In [76]:
allfour_reduce.rename(columns={'uuid':'Quantity'}, inplace=True)

In [77]:
series_group = allfour_reduce.groupby(['Country', 'Job_Title','age'])['Quantity'].count()

In [78]:
df_cuantity = pd.DataFrame(series_group).reset_index()

In [79]:
df_cuantity

,Country,Job_Title,age,Quantity
0,Austria,Automatic Data Processing Customer Liaison (AD...,31,1
1,Austria,Automatic Data Processing Customer Liaison (AD...,40,1
2,Austria,Automatic Data Processing Planner,48,1
3,Austria,Automatic Data Processing Planner,57,1
4,Austria,Automatic Data Processing Systems Security Spe...,27,1
...,...,...,...,...
6135,UnitedKingdom,unemployed,61,4
6136,UnitedKingdom,unemployed,62,10
6137,UnitedKingdom,unemployed,63,9
6138,UnitedKingdom,unemployed,64,6
